In [5]:
%load_ext autoreload
%autoreload 2


%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%cd C:\MAD4AG

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import random

warnings.filterwarnings('ignore')

C:\MAD4AG


In [7]:
combined_df= pd.read_pickle(r'.\dbs\twins\multiple_days_03-19.pkl')


df_clusters=pd.read_parquet(f'./dbs/intermediate/df_selected_clusters.parquet')

df_clusters = df_clusters[df_clusters.uid.isin(combined_df.uid)]


In [8]:
# calculate mean and median of the number_of_other_cluster in MAD data


df_clusters_statistic= df_clusters[df_clusters.act_type=='other'].groupby(['uid']).size().reset_index(name='number_of_other_cluster')

df_clusters_statistic = pd.merge(combined_df[['uid']], df_clusters_statistic, on='uid', how='left')
df_clusters_statistic.fillna(0, inplace=True)

print('mean: ', df_clusters_statistic.number_of_other_cluster.mean())
print('median: ', df_clusters_statistic.number_of_other_cluster.median())
print('the share of people more than 10 other clusters: ',len(df_clusters_statistic[df_clusters_statistic.number_of_other_cluster>10])/len(df_clusters_statistic))
df_clusters_statistic.number_of_other_cluster.value_counts()

mean:  6.574893589054653
median:  3.0
the share of people more than 10 other clusters:  0.18115956036139128


1.0      57089
2.0      43543
3.0      28524
4.0      20042
5.0      15291
         ...  
142.0        1
331.0        1
137.0        1
139.0        1
146.0        1
Name: number_of_other_cluster, Length: 161, dtype: int64

In [9]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)
df = df[df.holiday_s != 1]
df = df[df.weekday_s == 1]
df = df.drop(['holiday_s', 'weekday_s'], axis=1)



In [10]:
df_clusters_freq = df.groupby(['uid','cluster']).size().reset_index(name='clusters_freq')

In [11]:
df_clusters = pd.merge(df_clusters, df_clusters_freq, on=['uid','cluster'], how='left')

In [12]:
df_clusters_other = df_clusters[df_clusters.act_type=='other']

In [13]:
df_clusters_other['freq_sum'] = df_clusters_other.groupby(['uid'])['clusters_freq'].transform("sum")

In [14]:
df_clusters_other['other_prob'] = df_clusters_other['clusters_freq']/df_clusters_other['freq_sum']

In [15]:
df_clusters_list= df_clusters_other.groupby(['uid'])['cluster'].apply(list).reset_index(name='clusters')

df_other_prob_list = df_clusters_other.groupby(['uid'])['other_prob'].apply(list).reset_index(name='other_prob')

In [16]:
df_other = pd.merge(combined_df[['uid', 'other_count_0', 'other_count_1', 'other_count_2', 'other_count_3', 'other_count_4']],  df_clusters_list, on='uid', how='left')
df_other = pd.merge(df_other, df_other_prob_list, on='uid', how='left')

In [17]:
df_other = df_other.dropna(subset=['clusters'])

## sample other activities

In [18]:
# def other_finder(data , i):
#     other_clusters_place = np.argmax(np.random.multinomial(1, data['other_prob'], data['other_count_'+ str(i)]), axis=1).tolist()
#     clusters_list = data['clusters']
#     other_clusters = [clusters_list[place] for place in other_clusters_place]
#     return other_clusters

In [19]:
def other_finder(data , i):
    clusters_list = data['clusters']
    sample_size = data['other_count_'+ str(i)]
    if len(clusters_list)>= sample_size:
        other_clusters = np.random.choice(clusters_list, sample_size, p=data['other_prob'], replace=False).tolist()
    else:
        other_clusters = np.random.choice(clusters_list, len(clusters_list), p=data['other_prob'], replace=False).tolist()
        other_clusters += np.random.choice(clusters_list, sample_size-len(clusters_list), p=data['other_prob'], replace=True).tolist()
    random.shuffle(other_clusters)

    return other_clusters

In [20]:
for i in range(5):
    column_name= 'other_clusters_'+str(i)
    df_other[column_name] = df_other.apply(lambda row: other_finder(row, i), axis=1)

## show the selected other activities in the activity clusters dataframe

In [21]:
df_clusters_other_select=pd.merge(df_clusters_other[['uid','cluster']], df_other[['uid','other_clusters_0','other_clusters_1','other_clusters_2','other_clusters_3','other_clusters_4']], on='uid', how='left')

In [22]:
#df_clusters5.dropna(subset=['other_clusters_0'],inplace=True)

In [23]:
def other_selection(data, i):
    if data['cluster'] in data['other_clusters_' + str(i)]:
        selection='1'
    else:
        selection='0'
    return selection



for i in range(5):
    column_name = 'selection_' + str(i)
    df_clusters_other_select[column_name] = df_clusters_other_select.apply(lambda row: other_selection(row, i), axis=1)

In [24]:
# df_clusters = pd.merge(df_clusters, df_clusters_other_select[['uid','cluster','selection_0','selection_1','selection_2','selection_3','selection_4']], on= ['uid','cluster'], how='left' )

In [25]:
df_clusters = pd.merge(df_clusters, df_clusters_other_select, on= ['uid','cluster'], how='left' )

In [26]:
df_clusters['selection_0'][df_clusters.act_type=='home']='Home'
df_clusters['selection_1'][df_clusters.act_type=='home']='Home'
df_clusters['selection_2'][df_clusters.act_type=='home']='Home'
df_clusters['selection_3'][df_clusters.act_type=='home']='Home'
df_clusters['selection_4'][df_clusters.act_type=='home']='Home'

In [27]:
df_clusters['selection_0'][df_clusters.act_type=='work']='Work'
df_clusters['selection_1'][df_clusters.act_type=='work']='Work'
df_clusters['selection_2'][df_clusters.act_type=='work']='Work'
df_clusters['selection_3'][df_clusters.act_type=='work']='Work'
df_clusters['selection_4'][df_clusters.act_type=='work']='Work'

In [28]:
df_clusters['selection_0'][df_clusters.act_type=='school']='School'
df_clusters['selection_1'][df_clusters.act_type=='school']='School'
df_clusters['selection_2'][df_clusters.act_type=='school']='School'
df_clusters['selection_3'][df_clusters.act_type=='school']='School'
df_clusters['selection_4'][df_clusters.act_type=='school']='School'

In [29]:
df_clusters.loc[:, ['selection_0','selection_1','selection_2','selection_3','selection_4']] = df_clusters.loc[:, ['selection_0','selection_1','selection_2','selection_3','selection_4']].replace({'1': 'Other','0': 'non'})

In [30]:
filename = r'.\dbs\intermediate\df_selected_clusters_activities.parquet'

df_clusters.to_parquet(filename)